# CIFAR-100 A/B/C/D/E Benchmark (Colab)

This notebook runs CIFAR-100 experiments with param-matched A/B/C/D/E models using our MoP project.

- A: Baseline ViT-style attention
- B: MoP
- C: Cross-View Mixer
- D: Multi-Hop
- E: Edgewise (per-edge CNN gates)

It uses the full CIFAR-100 dataset via `torchvision` and saves results under `results/`.



In [ ]:
# Setup: clone and install
!pip -q install -U pip
!if [ ! -d MoP ]; then git clone https://github.com/Eran-BA/MoP.git ; fi
%cd MoP/
!pip -q install -r requirements.txt

import torch, torchvision
print('Torch:', torch.__version__, 'CUDA available:', torch.cuda.is_available())



In [ ]:
# Quick sanity: CIFAR-100 is available in torchvision, no manual download needed.
from torchvision import datasets
from torchvision import transforms

transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

train = datasets.CIFAR100(root='/content/data', train=True, download=True, transform=transform)
val   = datasets.CIFAR100(root='/content/data', train=False, download=True, transform=transforms.ToTensor())
print('Train size:', len(train), 'Val size:', len(val))



In [ ]:
# Run A/B/C/D/E at ~5M params (single seed)
!python experiments/cifar100_ab5_param_budgets.py \
  --targets 5000000 --models A B C D E \
  --steps 20000 --eval_every 200 --batch 256 \
  --ew_views 5 --ew_use_k3 --ew_share_qkv \
  --xview_transpose --xview_t1 0.2 --xview_t2 0.2 --xview_enable_prior --xview_prior_weight 0.5 --xview_anchor_mode argmax_row_sum \
  --mh_hops 3 --mh_gate_chain 1.0



In [ ]:
# Run A/B/C/D/E at ~50M params (single seed)
!python experiments/cifar100_ab5_param_budgets.py \
  --targets 50000000 --models A B C D E \
  --steps 60000 --eval_every 500 --batch 256 \
  --lr_large 0.001 --warmup_frac 0.1 \
  --ew_views 5 --ew_use_k3 --ew_share_qkv \
  --xview_transpose --xview_t1 0.2 --xview_t2 0.2 --xview_enable_prior --xview_prior_weight 0.5 --xview_anchor_mode argmax_row_sum \
  --mh_hops 3 --mh_gate_chain 1.0



In [ ]:
# Multi-seed tournament (5 seeds) at ~5M (plan_only example)
!python experiments/ab5_tournament.py \
  --dataset cifar100 --targets 5000000 --seeds 0 1 2 3 4 \
  --plan_only --out_dir results/cifar100_tournament_plan



In [ ]:
# Generate CIFAR-100 paper tables
!python experiments/ab5_paper_benchmark.py --datasets cifar100
!ls -la results/paper_benchmark

